## Learning Dyanmics and Lyapunov function for Van der Pol

In [ ]:
# To install dReal, if you use Google CoLab
# If not CoLab, command out
import pkgutil
if not pkgutil.find_loader("dreal"):
  !curl https://raw.githubusercontent.com/dreal/dreal4/master/setup/ubuntu/18.04/install.sh | bash
  !pip install dreal --upgrade

In [ ]:
# -*- coding: utf-8 -*-
from dreal import *
from Functions import *
import torch 
import torch.nn.functional as F
import numpy as np
import timeit 
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Actual dynamical system

In [ ]:
def f_value(x):
    y = torch.zeros_like(x)
    y[:,0] = - x[:,1]
    y[:,1] = x[:,0] + (x[:,0]**2-1)*x[:,1]
    return y

## NN for learning the dynamics f

In [ ]:
# NN for learning dynamics
class fNet(torch.nn.Module):
    def __init__(self,n_input, n_hidden1,  n_output):
        super().__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden1)
        self.layer2 = torch.nn.Linear(n_hidden1,n_output)   
        self.to(device)

    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = self.layer2(h_1) 
        return out

## Learning the dynamics with NNs

In [ ]:
# generate training dataset
N_f = 3001
xx = np.linspace(-1.5,1.5, N_f, dtype = float)
x_f = []
for i in range(0,N_f): 
    for j in range(0,N_f):
        x_f.append([xx[j],xx[i]])

x_f = torch.tensor(x_f)
x_f = x_f.float()

# target
t_f = f_value(x_f)  # actual dynamics
t_f = t_f.float()

In [ ]:
# define parameters
max_iter = 1000
losses = []
# NN: 1 hidden layers with 100 neurons each layer 
fnet = fNet(n_input = 2, n_hidden1 = 100,  n_output = 2)
optimizer = torch.optim.Adam(fnet.parameters(), lr = 0.1)

loss_func = torch.nn.MSELoss(reduction='sum')

for epoch in tqdm(range(max_iter)):
    x_f = x_f.to(device)
    t_f = t_f.to(device)
    y_nn = fnet(x_f)
    loss = loss_func(y_nn,t_f)
    losses.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        torch.cuda.empty_cache()

plt.plot(losses)

In [ ]:
# clear cache
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
# train more epoches if needed as we need alpha to be very small
losses = []
optimizer = torch.optim.Adam(fnet.parameters(), lr = 0.001) # 
loss_func = torch.nn.MSELoss(reduction='sum')
# fnet = fnet.to(device)

for epoch in tqdm(range(50)):
    x_f = x_f.to(device)
    t_f = t_f.to(device)
    y_nn = fnet(x_f)
    loss = loss_func(y_nn,t_f)
    losses.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        torch.cuda.empty_cache()

plt.plot(losses)

In [ ]:
losses[-1]

In [ ]:
# generate testing dataset
r = 1.2 # region of interest
N_l = 2401
xl = np.linspace(-r,r,N_l, dtype = float)
x_l = []
for i in range(0,N_l): 
    for j in range(0,N_l):
        x_l.append([xl[j],xl[i]])

x_l = torch.tensor(x_l)
x_l = x_l.float()

# target
t_l = f_value(x_l)  # actual dynamics
t_l = t_l.float()

In [ ]:
# output of FNN
x_l = x_l.to(device)
t_l = t_l.to(device)
y_l = fnet(x_l)

# maximum of loss
loss_all = torch.norm(y_l-t_l, dim = 1)
alpha = torch.max(loss_all)

In [ ]:
# save the weights to calculate the Lipschitz constant with LipSDP
f_w1 = fnet.layer1.weight.data.cpu().numpy()
f_w2 = fnet.layer2.weight.data.cpu().numpy()

np.savetxt("fw1.txt", f_w1, fmt="%s")
np.savetxt("fw2.txt", f_w2, fmt="%s")

In [ ]:
# # save the fNN
# torch.save(fnet.cpu(), 'VDP_fnet.pt')

# # load fNN from a file
# fnet = torch.load('VDP_fnet.pt').to(device)

## Plot the approximated results

In [ ]:
# dataset for plot
N_p = 300
xp = np.linspace(-r,r,N_p, dtype = float).reshape(N_p,1)
xx = np.linspace(-r,r,N_p, dtype = float).reshape(N_p,1)

for n in range(1): # for dim n
    xp = np.concatenate((xp,xx), axis=1 ) 

xp = torch.tensor(xp)
Xp = xp.float()


In [ ]:
# fnet_1 = fnet.to('cpu') # to plot, command out this line
yp = fnet_1(Xp)
tp = f_value(xp)

with torch.no_grad():
    plt.figure(1)
    plt.plot(xp[:,0], yp[:,0],label='NN')
    plt.plot(xp[:,0], tp[:,0], label='Actual')
    plt.xlabel('x1 & x2 pair')
    plt.ylabel('$\dot{x_1}$')
    plt.legend(loc = 0)

    plt.figure(2)
    plt.plot(xp[:,0],yp[:,1],label='NN')
    plt.plot(xp[:,0],tp[:,1],label='Aactual')
    plt.xlabel('x1 & x2 pair')
    plt.ylabel('$\dot{x_2}$')
    plt.legend()

## Learned dynamics

In [ ]:
def f_learned(x):
    y = fnet(x)
    return y

## Neural network model for Lyapunov function V

In [ ]:
class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output):
        super(Net, self).__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden)
        self.layer2 = torch.nn.Linear(n_hidden,n_output)
        self.to(device) 
        
    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = sigmoid(self.layer2(h_1))
        return out

## Parameters

In [ ]:
'''
For learning 
'''
N = 500            # sample size
D_in = 2            # input dimension
H1 = 6              # hidden dimension
D_out = 1           # output dimension
torch.manual_seed(10)  

x = torch.Tensor(N, D_in).uniform_(-r, r)           
x_0 = torch.zeros([1, 2])
x_0 = x_0.to(device)

'''
For verifying 
'''
x1 = Variable("x1")
x2 = Variable("x2")
vars_ = [x1,x2]
config = Config()
config.use_polytope_in_forall = True
config.use_local_optimization = True
config.precision = 1e-2
beta = -0.02 # initial guess of beta
# Checking candidate V within a ball around the origin (ball_lb ≤ sqrt(∑xᵢ²) ≤ ball_ub)
ball_lb = 0.2
ball_ub = 1.2

# parameters for beta
Kf = 3.4599
KF = 5.452
d = 5e-4
loss = 0.0085 # equals alpha above

## Learning and Falsification

In [ ]:
out_iters = 0
valid = False
while out_iters < 2 and not valid: 
    start = timeit.default_timer()
    model = Net(D_in,H1, D_out)
    L = []
    i = 0 
    t = 0
    max_iters = 3000 # increase number of epoches if cannot find a valid LF
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # learned dynamics
    f_w1 = fnet.layer1.weight.data.cpu().numpy()
    f_w2 = fnet.layer2.weight.data.cpu().numpy()
    f_b1 = fnet.layer1.bias.data.cpu().numpy()
    f_b2 = fnet.layer2.bias.data.cpu().numpy()

    f_h1 = []
    f_z1 = np.dot(vars_,f_w1.T)+f_b1
    for n in range(len(f_z1)):
        f_h1.append(tanh(f_z1[n]))

    f_learn = np.dot(f_h1,f_w2.T)+f_b2

    while i < max_iters and not valid: 
        x = x.float()
        x = x.to(device)
        V_candidate = model(x)
        X0 = model(x_0)
        f = f_learned(x)
        Circle_Tuning = Tune(x)
        Circle_Tuning = Circle_Tuning.to(device)
        # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
        L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0)
        
        dVdx = torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight)

        # With tuning
        Lyapunov_risk = (F.relu(-V_candidate)+ 1.2*F.relu(L_V+0.2)).mean()\
                    +((Circle_Tuning-V_candidate).pow(2)).mean()+ 1.2*(X0).pow(2) + 0.01*torch.norm(dVdx) 


        print(i, "Lyapunov Risk=",Lyapunov_risk.item()) 
        L.append(Lyapunov_risk.item())
        optimizer.zero_grad()
        Lyapunov_risk.backward()
        optimizer.step() 

        # save the weights and biases 
        w1 = model.layer1.weight.data.cpu().numpy()
        w2 = model.layer2.weight.data.cpu().numpy()
        b1 = model.layer1.bias.data.cpu().numpy()
        b2 = model.layer2.bias.data.cpu().numpy()
        
        # Falsification with SMT solver
        if i % 10 == 0:
            
            # Candidate V
            z1 = np.dot(vars_,w1.T)+b1

            a1 = []
            for j in range(0,len(z1)):
                a1.append(tanh(z1[j]))
            z2 = np.dot(a1,w2.T)+b2
            V_learn = tanh(z2.item(0))

            print('===========Verifying==========')        
            start_ = timeit.default_timer() 
            beta = -np.maximum(beta, -0.02) # in case beta is too negative and cannot return any results
            result= CheckLyapunov(vars_, f_learn, V_learn, ball_lb, ball_ub, config, beta) # SMT solver
            stop_ = timeit.default_timer() 

            if (result): 
                print("Not a Lyapunov function. Found counterexample: ")
                print(result)
                x = x.to('cpu')
                x = AddCounterexamples(x,result,10)
            else:  
                # calculate norm of dVdx with the SMT solver
                M = 0.05 # lower bound of M
                violation = CheckdVdx(vars_, V_learn, ball_ub, config, M) 
                while violation:
                    violation = CheckdVdx(vars_, V_learn, ball_ub, config, M)
                    if not violation:
                        dvdx_bound = np.sqrt(M)
                        print(dvdx_bound, "is the norm of dVdx")
                    M += 0.001
                beta = -dvdx_bound*((Kf+KF)*d+loss) # update beta 
                result_strict= CheckLyapunov(vars_, f_learn, V_learn, ball_lb, ball_ub, config, beta) # SMT solver
                if not result_strict:
                    valid = True
                    print("Satisfy conditions with beta = ", beta)
                    print(V_learn, " is a Lyapunov function.")
            t += (stop_ - start_)
            print('==============================') 
        i += 1

    stop = timeit.default_timer()


    np.savetxt("w1_vdp.txt", model.layer1.weight.data.cpu(), fmt="%s")
    np.savetxt("w2_vdp.txt", model.layer2.weight.data.cpu(), fmt="%s")
    np.savetxt("b1_vdp.txt", model.layer1.bias.data.cpu(), fmt="%s")
    np.savetxt("b2_vdp.txt", model.layer2.bias.data.cpu(), fmt="%s")

    print('\n')
    print("Total time: ", stop - start)
    print("Verified time: ", t)
    
    out_iters+=1

### Checking result with bounded beta if needed

In [ ]:
beta = -0.02
start_ = timeit.default_timer() 
result = CheckLyapunov(vars_, f_learn, V_learn, ball_lb, ball_ub, config, beta)
stop_ = timeit.default_timer() 

if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
    print(result)
else:  
    print("Satisfy conditions with beta = ",beta)
    print(V_learn, " is a Lyapunov function.")
t += (stop_ - start_)

## Checking with beta = 0 for actual dynamics

In [ ]:
beta = 0
f = [-x2,
        x1+(x1**2 - 1)*x2]
start_ = timeit.default_timer() 
result = CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config, beta)
stop_ = timeit.default_timer() 
if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
    print(result)
else:  
    print("Satisfy conditions with beta= ", beta)
    print(V_learn, " is a Lyapunov function for actual dynamics.")
t += (stop_ - start_)
